# List Guilds ID's with Names and Description

In [ ]:
import glob
import os
from pprint import pprint
import sqlite3
import json
from dotenv import load_dotenv
load_dotenv()

# Specify the directory path and the pattern to match all files (e.g., * for all files)

directory_path = os.environ.get("guilds_out")
pattern = '*.sqlite'

In [ ]:
print(directory_path)

In [ ]:
# LIST_GUILDS

sqlite_paths = glob.glob(os.path.join(directory_path, pattern))
pprint(sqlite_paths)
for file in sqlite_paths:
    pprint(file)
    tmp_conn = sqlite3.connect(file)
    tmp_cursor = tmp_conn.cursor()
    query = '''
        SELECT
            json_extract(raw_json, '$.id') as guild_id,
            json_extract(raw_json, '$.name') as guild_name
        FROM
            raw_guilds_t
        LIMIT 1
    '''
    tmp_cursor.execute(query)
    data = tmp_cursor.fetchall()
    pprint(data)
    tmp_cursor.close()
    tmp_conn.close()
    print("\n")

In [ ]:
# Connect to the SQLite database file

sqlite_path = sqlite_paths[0] #os.environ.get("db_url")
pprint(sqlite_path)
conn = sqlite3.connect(sqlite_path)

# Create a cursor object
cursor = conn.cursor()

In [ ]:
# Execute an SQL query
cursor.execute("""SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';""")

# Fetch all the data
data = cursor.fetchall()

# Iterate through the data and print it
pprint(data)

In [ ]:
# Execute an SQL query
cursor.execute("""SELECT * FROM raw_messages_t LIMIT 5;""")

# Fetch all the data
data = cursor.fetchall()

# Iterate through the data and print it
pprint(data)

In [ ]:
# LIST_USERS

query = '''
SELECT
    DISTINCT( json_extract(raw_json, '$.id') ) as author_id,
    json_extract(raw_json, '$.name') as name,
    json_extract(raw_json, '$.is_bot') as is_bot
FROM
    raw_authors_t
LIMIT 5
'''
cursor.execute(query)
users = cursor.fetchall()
pprint(len(users))
pprint(users[0:5])

In [ ]:
# LIST_USERS_WITH_ROLES

query = '''
SELECT
    DISTINCT
    user_id,
    usenamer_id,
    json_extract(raw_roles_t.raw_json, '$.id'),
    json_extract(raw_roles_t.raw_json, '$.name')
FROM 
    raw_roles_t
JOIN (
    SELECT
        DISTINCT( json_extract(raw_authors_t.raw_json, '$.id') ) as user_id,
        json_extract(raw_authors_t.raw_json, '$.name') as usenamer_id,
        json_extract(raw_authors_t.raw_json, '$.isBot') as isBot
    FROM
        raw_authors_t
    ) as authors_t
ON
    authors_t.user_id = 
    json_extract(raw_roles_t.raw_json, '$.user_id.id')
'''
cursor.execute(query)
users = cursor.fetchall()
pprint(len(users))
pprint(users[0:5])

In [ ]:
# LIST_CHANNELS

query = '''
SELECT
    DISTINCT(json_extract(raw_json, '$.id')) as channel_id,
    (json_extract(raw_json, '$.type')) as type,
    (json_extract(raw_json, '$.category')) as category,
    (json_extract(raw_json, '$.name')) as name
FROM
    raw_channels_t
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(len(data))
pprint(data[0:5])

# NUMBER OF MESSAGES

In [ ]:
# NUM_MESSAGES_IN_GUILD

query = '''
SELECT COUNT(*) as data FROM raw_messages_t
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data[0][0])

In [ ]:
# NUM_MESSAGES_PER_USER

query = '''

SELECT
    json_extract(raw_authors_t.raw_json, '$.id') as author_id,
    json_extract(raw_authors_t.raw_json, '$.name') as author_name,
    message_count.count as message_count
FROM raw_authors_t
JOIN (
    SELECT
        DISTINCT( json_extract(raw_messages_t.raw_json, '$.author_id') ) as author_id,
        count(*) as count
    FROM raw_messages_t
    GROUP BY author_id
) AS message_count
ON json_extract(raw_authors_t.raw_json, '$.id') = author_id
GROUP BY author_id
ORDER BY count DESC
LIMIT 20

'''

cursor.execute(query)
data = cursor.fetchall()
pprint(data)
user_for_queries = data[0][0]
print(f"\nuser_for_queries = {user_for_queries}\n")
pprint(data[:20])


In [ ]:
# NUM_MESSAGES_PER_CHANNEL

query = f'''
    SELECT
        DISTINCT( json_extract(raw_messages_t.raw_json, '$.channel_id') ) as channel_id,
        count(*) as count
    FROM raw_messages_t
    WHERE  json_extract(raw_messages_t.raw_json, '$.author_id') = '{user_for_queries}'
    GROUP BY channel_id
'''
print(query)
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

In [ ]:
# NUM_MESSAGES_PER_CHANNEL

query = f'''
SELECT
    json_extract(raw_channels_t.raw_json, '$.name') as channel_name,
    json_extract(raw_channels_t.raw_json, '$.id') as channel_id,
    message_count.count as message_count
FROM raw_channels_t
JOIN (
    SELECT
        DISTINCT( json_extract(raw_messages_t.raw_json, '$.channel_id') ) as channel_id,
        count(*) as count
    FROM raw_messages_t
    WHERE  json_extract(raw_messages_t.raw_json, '$.author_id') = '{user_for_queries}'
    GROUP BY channel_id
) AS message_count
ON json_extract(raw_channels_t.raw_json, '$.id') = channel_id
ORDER BY count DESC
'''
print(query)
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

In [ ]:
# NUM_MESSAGES_PER_CHANNEL_FOR_USER

query = f'''
    SELECT
        DISTINCT( json_extract(raw_messages_t.raw_json, '$.channel_id') ) as channel_id,
        count(*) as count
    FROM raw_messages_t
    WHERE  json_extract(raw_messages_t.raw_json, '$.author_id') = '{user_for_queries}'
    GROUP BY channel_id
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

In [ ]:
# NUM_MESSAGES_FROM_USER_PER_CHANNEL

query = f'''
SELECT
    json_extract(raw_channels_t.raw_json, '$.id') as channel_id,
    json_extract(raw_channels_t.raw_json, '$.name') as channel_name,
    message_count.count as message_count
FROM raw_channels_t
JOIN (
    SELECT
        DISTINCT( json_extract(raw_messages_t.raw_json, '$.channel_id') ) as channel_id,
        count(*) as count
    FROM raw_messages_t
    WHERE json_extract(raw_messages_t.raw_json, '$.author') = '{user_for_queries}'
    GROUP BY channel_id
) AS message_count
ON json_extract(raw_channels_t.raw_json, '$.id') = channel_id
ORDER BY count DESC
'''
cursor.execute(query)
data = cursor.fetchall()
channel_for_queries = data[0][0]
print(f"\nchannel_for_queries = {channel_for_queries}\n")
pprint(data)

# LIST CHANNELS

In [ ]:
# LIST_CHANNELS_USER_POSTED_IN

query = f'''

SELECT
    DISTINCT (json_extract(raw_json, '$.channel_id'))
FROM raw_messages_t
WHERE json_extract(raw_json, '$.author_id') = '{user_for_queries}'
LIMIT 10
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

In [ ]:
# LIST_CHANNELS_BY_OLDEST_MESSAGE

query = f'''

SELECT
    DISTINCT (json_extract(raw_messages_t.raw_json, '$.channel_id')) as channel_id,
    MIN (json_extract(raw_messages_t.raw_json, '$.timestamp')) as timestamp,
    channels_t.name as channel_name
FROM raw_messages_t
JOIN (
    SELECT 
        DISTINCT ( json_extract(raw_channels_t.raw_json, '$.id') ) as channel_id2,
        json_extract(raw_channels_t.raw_json, '$.name') as name
    FROM raw_channels_t
) as channels_t
ON channels_t.channel_id2 = channel_id
GROUP BY channel_id
ORDER BY timestamp ASC
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

# LONGEST MESSAGES

In [ ]:
# LONGEST_MESSAGES_PER_GUILD

query = '''

SELECT
    json_extract(raw_json, '$.content') as msg_content,
    LENGTH( json_extract(raw_json, '$.content') ) as msg_length
FROM raw_messages_t
ORDER BY msg_length DESC
LIMIT 100

'''
cursor.execute(query)
data = cursor.fetchall()
for message in data[:3]:
    print(message[0])
    print("\n\n\n================\n\n\n")

In [ ]:
# LONGEST_MESSAGES_PER_USER

query = f'''

SELECT
    json_extract(raw_json, '$.content') as msg_content,
    LENGTH( json_extract(raw_json, '$.content') ) as msg_length
FROM raw_messages_t
WHERE json_extract(raw_json, '$.author') = '{user_for_queries}'
ORDER BY msg_length DESC
LIMIT 100

'''
cursor.execute(query)
data = cursor.fetchall()
print(len(data))
for message in data[:10]:
    print(message[0])
    print("\n\n\n================\n\n\n")

In [ ]:
# LONGEST_MESSAGES_PER_CHANNEL

query = f'''

SELECT msg_count, COUNT(*) as count_of_msg_count
FROM (
    SELECT json_extract(raw_json, '$.author') as msg_count
    FROM raw_messages_t
    WHERE json_extract(raw_json, '$.channel_id') = '{channel_for_queries}'
) AS subquery
GROUP BY msg_count
ORDER BY count_of_msg_count DESC
LIMIT 5;

'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

In [ ]:
# LONGEST_MESSAGES_PER_USER_IN_CHANNEL

query = f'''

SELECT
    json_extract(raw_json, '$.content') as msg_content,
    LENGTH( json_extract(raw_json, '$.content') ) as msg_length
FROM raw_messages_t
WHERE json_extract(raw_json, '$.channel_id') = '585085189823266875'
    AND json_extract(raw_json, '$.author_id') = '{user_for_queries}'
ORDER BY msg_length DESC
LIMIT 100

'''
cursor.execute(query)
data = cursor.fetchall()
for message in data[:5]:
    print(message[0])
    print("\n\n\n================\n\n\n")

# GET_ORDERED_MESSAGES

In [ ]:
# GET_MESSAGES_FROM_GUILD

query = '''

SELECT
    json_extract(raw_json, '$.content') as msg_content
FROM raw_messages_t
ORDER BY json_extract(raw_json, '$.timestamp') ASC
LIMIT 100

'''
cursor.execute(query)
data = cursor.fetchall()
for message in data[:5]:
    print(message[0])
    print("\n\n\n================\n\n\n")


In [ ]:
# GET_MESSAGES_FROM_CHANNEL

query = f'''

SELECT
    json_extract(raw_json, '$.content') as msg_content
FROM raw_messages_t
WHERE json_extract(raw_json, '$.channel_id') = '{channel_for_queries}'
ORDER BY json_extract(raw_json, '$.timestamp') ASC
LIMIT 100

'''
cursor.execute(query)
data = cursor.fetchall()
print(channel_for_queries)
pprint(data)
for message in data[:5]:
    print(message[0])
    print("\n\n\n================\n\n\n")


In [ ]:
# GET_MESSAGES_FROM_USER

query = f'''

SELECT
    json_extract(raw_json, '$.content') as msg_content
FROM raw_messages_t
WHERE json_extract(raw_json, '$.author') = '{user_for_queries}'
ORDER BY json_extract(raw_json, '$.timestamp') ASC
LIMIT 100

'''
cursor.execute(query)
data = cursor.fetchall()
for message in data[:5]:
    print(message[0])
    print("\n\n\n================\n\n\n")


In [ ]:
# GET_ORDERED_MESSAGES_FROM_USER_IN_CHANNEL

query = f'''

SELECT
    json_extract(raw_json, '$.content') as msg_content
FROM raw_messages_t
WHERE json_extract(raw_json, '$.channel_id') = '{channel_for_queries}'
    AND json_extract(raw_json, '$.author') = '{user_for_queries}'
ORDER BY json_extract(raw_json, '$.timestamp') ASC
LIMIT 100

'''
cursor.execute(query)
data = cursor.fetchall()
for message in data[:5]:
    print(message[0])
    print("\n\n\n================\n\n\n")


# Average Message Length

In [ ]:
# AVERAGE_MESSAGE_LENGTH_PER_GUILD

query = '''

SELECT
    AVG( LENGTH( json_extract(raw_json, '$.content') ) )
FROM raw_messages_t

'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

In [ ]:
# AVERAGE_MESSAGE_LENGTH_FOR_USER

query = f'''

SELECT
    AVG( LENGTH( json_extract(raw_json, '$.content') ) )
FROM raw_messages_t
WHERE json_extract(raw_json, '$.author') = '{user_for_queries}'
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

In [ ]:
# AVERAGE_MESSAGE_LENGTH_PER_USER

query = '''
SELECT
    json_extract(raw_json, '$.name') as name,
    author,
    avg_length
FROM raw_authors_t
JOIN (
    SELECT
        json_extract(raw_json, '$.author') as author,
        AVG( LENGTH( json_extract(raw_json, '$.content') ) ) avg_length
    FROM raw_messages_t
    GROUP BY author
    ORDER BY avg_length DESC
    LIMIT 10
    ) AS subquery
ON json_extract(raw_authors_t.raw_json, '$.id') = author
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

In [ ]:
# AVERAGE_MESSAGE_LENGTH_PER_CHANNEL

query = f'''
SELECT
    json_extract(raw_json, '$.name') as name,
    avg_length
FROM raw_channels_t
JOIN (
    SELECT
        json_extract(raw_json, '$.channel_id') as channel_id,
        AVG( LENGTH( json_extract(raw_json, '$.content') ) ) avg_length
    FROM raw_messages_t
    GROUP BY channel_id
    ORDER BY avg_length DESC
    ) AS subquery
ON json_extract(raw_channels_t.raw_json, '$.id') = channel_id
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

In [ ]:
# AVERAGE_MESSAGE_LENGTH_PER_USER_IN_CHANNEL

query = f'''
SELECT
     json_extract(raw_channels_t.raw_json, '$.name') as channel_name,
     author_name,
     avg_length
FROM raw_channels_t JOIN
(
    SELECT
        DISTINCT( json_extract(raw_authors_t.raw_json, '$.name') ) as author_name,
        author,
        channel_id,
        avg_length
    FROM raw_authors_t
    JOIN (
        SELECT
            json_extract(raw_json, '$.author')  as author,
            json_extract(raw_json, '$.channel_id') as channel_id,
            AVG( LENGTH( json_extract(raw_json, '$.content') ) ) avg_length
        FROM raw_messages_t
            WHERE channel_id = '{channel_for_queries}'
        GROUP BY author
        ORDER BY avg_length DESC
        LIMIT 20
        ) AS subquery
    ON json_extract(raw_authors_t.raw_json, '$.id') = author
) ON json_extract(raw_channels_t.raw_json, '$.id') = channel_id
ORDER BY avg_length DESC
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

# SUM OF REACTIONS

In [ ]:
# SUM_OF_REACTIONS_PER_GUILD

query = '''

SELECT
    SUM(count) as reaction_count
FROM (
    SELECT
        json_extract(raw_json, '$.message_id') as message_id,
        json_extract(raw_json, '$.count') as count
    FROM
        raw_reactions_t
    ORDER BY count DESC
)

'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data[:10])

In [ ]:
# SUM_OF_REACTIONS_FOR_USER

query = '''

    SELECT
        *
    FROM
        raw_reactions_t
    LIMIT 5
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data[:10])

In [ ]:
# SUM_OF_REACTIONS_TO_USER

query = f'''

SELECT
    SUM(count) as reaction_count
FROM (
    SELECT
        json_extract(raw_json, '$.message_id') as message_id,
        json_extract(raw_json, '$.count') as count
    FROM
        raw_reactions_t
    WHERE json_extract(raw_json, '$.author_id') = '{user_for_queries}'
    ORDER BY count DESC
)

'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data[:10])

In [ ]:
# SUM_OF_REACTIONS_TO_EACH_USER

query = f'''

SELECT
    DISTINCT(  json_extract(raw_json, '$.name')  ) as name,
    reaction_count
FROM raw_authors_t
JOIN (
    SELECT
        author_id,
        SUM(count) as reaction_count
    FROM (
        SELECT
            json_extract(raw_json, '$.author_id') as author_id,
            json_extract(raw_json, '$.count') as count
        FROM
            raw_reactions_t
        ORDER BY count DESC
    )
    GROUP BY author_id
) ON json_extract(raw_json, '$.id') = author_id
ORDER BY reaction_count DESC

'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data[:10])

In [ ]:
# SUM_OF_REACTIONS_PER_CHANNEL

query = f'''
SELECT
    json_extract(raw_channels_t.raw_json, '$.name') as channel_name,
    reaction_count
FROM raw_channels_t JOIN
(
        SELECT
            json_extract(raw_json, '$.channel_id') as channel_id,
            SUM(  json_extract(raw_json, '$.count') ) as reaction_count
        FROM
            raw_reactions_t
        GROUP BY channel_id
) ON json_extract(raw_channels_t.raw_json, '$.id') = channel_id
ORDER BY reaction_count DESC

'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data[:20])

In [ ]:
# SUM_OF_REACTIONS_TO_USER_IN_CHANNEL


#TODO

# TODO
# Most Reacted to Message (Single Emoji)

In [ ]:
# Most Reacted to Message (Single Emoji) Per Guild

query = '''

SELECT
    json_extract(raw_json, '$.message_id') as message_id,
    json_extract(raw_json, '$.count') as count
FROM
    raw_reactions_t
ORDER BY count DESC


'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data[:10])

In [ ]:
# Sum of all reactions in Channel

query = '''

SELECT
    json_extract(raw_messages_t.raw_json, '$.id') as message_id,
    json_extract(raw_messages_t.raw_json, '$.author') as author,
    count,
    json_extract(raw_messages_t.raw_json, '$.content') as content
FROM raw_messages_t
JOIN
(
    SELECT 
        json_extract(raw_reactions_t.raw_json, '$.message_id') as message_id,
        json_extract(raw_json, '$.count') as count
    FROM
        raw_reactions_t
    ORDER BY count DESC
    LIMIT 25
) as reaction_messages
ON json_extract(raw_messages_t.raw_json, '$.id') = 
    reaction_messages.message_id

'''
cursor.execute(query)
data = cursor.fetchall()

In [ ]:
for result in data[:4]:
    print(f"message_id = {result[0]}")
    print(f"author = {result[1]}")
    print(f"reaction_count = {result[2]}")
    print(f"content... \n{result[3]}")
    print(f"\n\n==========\n\n")

In [ ]:
# Sum of all reactions in Channel

query = '''

SELECT
    json_extract(raw_messages_t.raw_json, '$.id') as message_id,
    json_extract(raw_messages_t.raw_json, '$.author') as author,
    count,
    json_extract(raw_messages_t.raw_json, '$.content') as content
FROM raw_messages_t
JOIN
( 
    SELECT
        json_extract(raw_reactions_t.raw_json, '$.message_id') as message_id,
        json_extract(raw_reactions_t.raw_json, '$.count') as count
    FROM
        raw_reactions_t
    GROUP BY count
    ORDER BY count DESC
    LIMIT 25
) as reaction_messages
ON json_extract(raw_messages_t.raw_json, '$.id') = 
    reaction_messages.message_id

'''
cursor.execute(query)
data = cursor.fetchall()

In [ ]:
for result in data[:4]:
    print(f"message_id = {result[0]}")
    print(f"author = {result[1]}")
    print(f"reaction_count = {result[2]}")
    print(f"content... \n{result[3]}")
    print(f"\n\n==========\n\n")

In [ ]:
# Sum of most reactions to messages in Channel

query = '''

SELECT
    json_extract(raw_messages_t.raw_json, '$.id') as message_id,
    json_extract(raw_messages_t.raw_json, '$.author') as author,
    count,
    json_extract(raw_messages_t.raw_json, '$.content') as content
FROM raw_messages_t
JOIN
( 
    SELECT
        json_extract(raw_reactions_t.raw_json, '$.message_id') as message_id,
        json_extract(raw_reactions_t.raw_json, '$.count') as count
    FROM
        raw_reactions_t
    GROUP BY count
    ORDER BY count DESC
    LIMIT 25
) as reaction_messages
ON json_extract(raw_messages_t.raw_json, '$.id') = 
    reaction_messages.message_id

'''
cursor.execute(query)
data = cursor.fetchall()

In [ ]:
for result in data[:4]:
    print(f"message_id = {result[0]}")
    print(f"author = {result[1]}")
    print(f"reaction_count = {result[2]}")
    print(f"content... \n{result[3]}")
    print(f"\n\n==========\n\n")

# TODO
# Most Reacted to Message (All Emoji)

In [ ]:
# Most reacted to Message All Emoji

query = '''

SELECT 
    message_id,
    SUM(COUNT) as total_count
FROM
(
    SELECT
        json_extract(raw_reactions_t.raw_json, '$.message_id') as message_id,
        json_extract(raw_reactions_t.raw_json, '$.count') as count
    FROM
        raw_reactions_t
)
GROUP BY message_id
order by total_count desc
LIMIT 25
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

In [ ]:
# Most reacted to Message in Channel All Emoji

query = '''

SELECT 
    message_id,
    SUM(COUNT) as total_count
FROM
(
    SELECT
        json_extract(raw_reactions_t.raw_json, '$.message_id') as message_id,
        json_extract(raw_reactions_t.raw_json, '$.count') as count
    FROM
        raw_reactions_t
)
GROUP BY message_id
order by total_count desc
LIMIT 25
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

# Popularity of Emoji

In [ ]:
# Most emoji used (Guild)

query = '''
SELECT 
    emoji_name,
    SUM(COUNT) as total_count
FROM
(
    SELECT
        json_extract(raw_json, '$.emoji.name') as emoji_name,
        json_extract(raw_json, '$.count') as count
    FROM
        raw_reactions_t
)
GROUP BY emoji_name
order by total_count desc
LIMIT 25
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

# Number of Messages per Day

In [ ]:
# Number of Messages per Day (Guild)
query = '''
SELECT strftime('%Y-%m-%d', DATETIME( json_extract(raw_json, '$.timestamp') )  ) AS date,
       COUNT(*) AS daily_sum
FROM raw_messages_t
GROUP BY date
ORDER BY date;
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data[:5])

In [ ]:
# Number of Messages per Day (User)
query = '''
SELECT strftime('%Y-%m-%d', DATETIME( json_extract(raw_json, '$.timestamp') )  ) AS date,
       COUNT(*) AS daily_sum
FROM raw_messages_t
WHERE json_extract(raw_json, '$.author') = '188027138891317249'
GROUP BY date
ORDER BY date;
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data[:5])

In [ ]:
# Number of Messages per Day (Channel)
query = f'''
SELECT strftime('%Y-%m-%d', DATETIME( json_extract(raw_json, '$.timestamp') )  ) AS date,
       COUNT(*) AS daily_sum
FROM raw_messages_t
WHERE json_extract(raw_json, '$.channel_id') = '{channel_for_queries}'
GROUP BY date
ORDER BY date;
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data[:5])

In [ ]:
# Number of Messages per Day (User in Channel)
query = f'''
SELECT strftime('%Y-%m-%d', DATETIME( json_extract(raw_json, '$.timestamp') )  ) AS date,
       COUNT(*) AS daily_sum
FROM raw_messages_t
WHERE json_extract(raw_json, '$.author') = '{user_for_queries}'
    AND json_extract(raw_json, '$.channel_id') = '{channel_for_queries}'
GROUP BY date
ORDER BY date;
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data[:5])

# Number of Messages based on Hour of Day

In [ ]:
# Number of Messages based on Hour of Day (Guild)
query = '''
SELECT strftime('%H', DATETIME(json_extract(raw_json, '$.timestamp'))  ) AS date,
       COUNT(*) AS daily_sum
FROM raw_messages_t
GROUP BY date
ORDER BY date;
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data)

# Weekends verses Weekdays

In [ ]:
average_messages_per_weekday_day = cursor.execute("""
SELECT 
(  select  
   count(*)
from raw_messages_t 
WHERE strftime('%w', DATETIME(json_extract(raw_json, '$.timestamp'))) == '1' 
OR    strftime('%w', DATETIME(json_extract(raw_json, '$.timestamp'))) == '2' 
OR    strftime('%w', DATETIME(json_extract(raw_json, '$.timestamp'))) == '3' 
OR    strftime('%w', DATETIME(json_extract(raw_json, '$.timestamp'))) == '4' 
OR    strftime('%w', DATETIME(json_extract(raw_json, '$.timestamp'))) == '5' ) 
/ 
(select  
   COUNT(  DISTINCT(  strftime('%Y-%m-%d', DATETIME(json_extract(raw_json, '$.timestamp')))  )  ) as num_days_of_week
from raw_messages_t 
WHERE strftime('%w', DATETIME(json_extract(raw_json, '$.timestamp'))) == '1' 
OR    strftime('%w', DATETIME(json_extract(raw_json, '$.timestamp'))) == '2' 
OR    strftime('%w', DATETIME(json_extract(raw_json, '$.timestamp'))) == '3' 
OR    strftime('%w', DATETIME(json_extract(raw_json, '$.timestamp'))) == '4' 
OR    strftime('%w', DATETIME(json_extract(raw_json, '$.timestamp'))) == '5' 
) as average_events_per_weekday
""").fetchone()

In [ ]:
average_messages_per_weekday_day

In [ ]:
average_events_per_weekend_day = cursor.execute("""
SELECT 
(  select  
   count(*)
from raw_messages_t 
WHERE strftime('%w', DATETIME(json_extract(raw_json, '$.timestamp'))) == '0' 
OR    strftime('%w', DATETIME(json_extract(raw_json, '$.timestamp'))) == '6'  ) 
/ 
(select  
   COUNT(  DISTINCT(  strftime('%Y-%m-%d', DATETIME(json_extract(raw_json, '$.timestamp')))  )  ) as num_days_of_week
from raw_messages_t 
WHERE strftime('%w', DATETIME(json_extract(raw_json, '$.timestamp'))) == '0' 
OR    strftime('%w', DATETIME(json_extract(raw_json, '$.timestamp'))) == '6'  
) as average_events_per_weekend_day
""").fetchone()

In [ ]:
average_events_per_weekend_day

# Median Message Length

In [ ]:
# Median Message Length (Guild)
query = '''
SELECT
  CASE
    WHEN COUNT(*) % 2 = 1 THEN
      CAST(MAX(length) AS REAL)
    ELSE
      (CAST(MAX(length) AS REAL) + CAST(MIN(length) AS REAL)) / 2
  END AS median_length
FROM (
  SELECT LENGTH(json_extract(raw_json, '$.content')) AS length
  FROM raw_messages_t
  WHERE LENGTH(json_extract(raw_json, '$.content')) > 0  -- Filter out empty strings if needed
  ORDER BY length
) subquery;
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data[:5])

In [ ]:
# Median Message Length (User)
query = f'''
SELECT
  CASE
    WHEN COUNT(*) % 2 = 1 THEN
      CAST(MAX(length) AS REAL)
    ELSE
      (CAST(MAX(length) AS REAL) + CAST(MIN(length) AS REAL)) / 2
  END AS median_length
FROM (
  SELECT LENGTH(json_extract(raw_json, '$.content')) AS length
  FROM raw_messages_t
  WHERE 
      json_extract(raw_json, '$.author') = '{user_for_queries}'
      AND LENGTH(json_extract(raw_json, '$.content')) > 0  -- Filter out empty strings if needed
  ORDER BY length
) subquery;
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data[:5])

In [ ]:
# Median Message Length (Channel)
query = f'''
SELECT
  CASE
    WHEN COUNT(*) % 2 = 1 THEN
      CAST(MAX(length) AS REAL)
    ELSE
      (CAST(MAX(length) AS REAL) + CAST(MIN(length) AS REAL)) / 2
  END AS median_length
FROM (
  SELECT LENGTH(json_extract(raw_json, '$.content')) AS length
  FROM raw_messages_t
  WHERE 
      json_extract(raw_json, '$.channel_id') = '{channel_for_queries}'
      AND LENGTH(json_extract(raw_json, '$.content')) > 0  -- Filter out empty strings if needed
  ORDER BY length
) subquery;
'''
cursor.execute(query)
data = cursor.fetchall()
pprint(data[:5])

In [ ]:
# Close the cursor and the database connection
# cursor.close()
# conn.close()